## Import Libs

In [1]:
import os
import json
import re
import ast
import base64
import io
from dataclasses import dataclass, field
from typing import List, Tuple
import random

In [2]:
import numpy as np
import tqdm
import torch

In [3]:
from transformers import BitsAndBytesConfig
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM
import datasets
from datasets import load_dataset, DatasetDict, Dataset
from huggingface_hub import login

In [4]:
!pip install --upgrade accelerate
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.0 MB/s eta 0:00:00


In [5]:
!pip install -q 'numpy<2' #datasets cohere torch transformers einops faiss-cpu
!pip install -q -U numpy==1.23.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.17 requires numpy>=1.24, but you have numpy 1.23.0 which is incompatible.
albumentations 1.4.17 requires numpy>=1.24.4, but you have numpy 1.23.0 which is incompatible.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 17.0.0 which is incompatible.
bayesian-optimization 1.5.1 requires numpy>=1.25, but you have numpy 1.23.0 which is incompatible.
blis 1.0.1 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.23.0 which is incompatible.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.23.0 which is incompatible.
chex 0.1.87 requires numpy>=1.24.1, but you have numpy 1.23.0 w

In [6]:
#import cohere
from kaggle_secrets import UserSecretsClient

In [7]:
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("COHERE_API_KEY")
secret_value_1 = user_secrets.get_secret("HF_KEY")

In [8]:
COHERE_MODEL = "/kaggle/input/aya-expanse/transformers/8b/1"
EMBEDDING_MODEL = 'jinaai/jina-embeddings-v3'
quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
GLOBAL_FESTIVALS_DATASET = 'ciol-research/global-festivals-wiki'
GLOBAL_FESTIVALS_DATASET_T = 'azminetoushikwasi/global-festivals-translated'
LANGAUGES = ['english', 'german', 'french', 'spanish', 'indian', 'japanese', 'korean', 'turkish', 'italian']

## Loading Models and APIs

In [9]:
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.7/249.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires urllib3<2.0.0, but you have urllib3 2.2.3 which is incompatible.
tensorflow 2.16.1 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11", but you have numpy 1.23.0 which is incompatible.
ydata-profiling 4.10.0 requires scipy<1.14,>=1.4.1, but you have scipy 1.14.1 which is incompatible.


In [10]:
import cohere
co = cohere.ClientV2(COHERE_MODEL)
